# TGDS Residual Learning

In [ ]:
import context
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import src.data.data_preprocessing as util
import src.models.lstm_model as lstm

tf.compat.v1.set_random_seed(1)

### Set Hyperparameters

In [ ]:
HYPER_PARAMS = {'n_epochs': 3,
                'use_case': 1,
                'd_sample': 1,
                'gauss_sigma': 10, 
                'feature_range_low': -1,
                'feature_range_high': 1,
                'n_steps': 50,
                'n_features': 2,
                'n_lstm_units_1': 50,
                'alpha_1': 0.1,
                'n_lstm_units_2': 20,
                'alpha_2': 0.1,
                'n_dense_units': 10,
                'activation_output_layer': 'tanh',
                'dropout': 0.2,
                'learning_rate': 0.001,
                'early_stopping': 20,
                'optimizer': 'Adam',
                'metric': 'mae',
                'loss_funcs': ['residual']
                }

### Prepare Training/Validation/Test Data
Training data can be modified to account for specific use cases. The prepare_data function is fed with the required profiles and will preprocess the sequence. Three use cases are predefined:
###### 1. Use Case: Train and Test data are built from equal profiles
 The purpose is to find out if the network is able to accurately reproduce the seen profiles
###### 2. Use Case: Train and Test data are of equal value range but different profiles
The purpose is to find out if the network can abstract onto unseen profiles with similar characteristics
###### 3. Use Case: Train and Test data are of different distributions
The purpose is to find out if the network can abstract onto new value ranges and unseen profiles

In [ ]:
train_profiles_usecase_1 = ['Profile 10A']
train_profiles_usecase_2 = ['Profile 10A']
train_profiles_usecase_3 = ['Profile 10A']

val_profiles_usecase_1 = ['Profile 10A']
val_profiles_usecase_2 = ['Profile 10A Run 040618']
val_profiles_usecase_3 = ['Profile 25A Run 040618']

test_profiles_usecase_1 = ['Profile 10A']
test_profiles_usecase_2 = ['Profile 10A Run 070618']
test_profiles_usecase_3 = ['Profile 25A Run 070618']

if HYPER_PARAMS['use_case'] == 1:
    # use case 1
    X_train, y_train, scalers_train = util.prepare_data(HYPER_PARAMS, train_profiles_usecase_1, 0, 4)
    X_validation, y_validation, scalers_validation = util.prepare_data(HYPER_PARAMS, val_profiles_usecase_1, 0, 4)
    X_test, y_test, scalers_test = util.prepare_data(HYPER_PARAMS, test_profiles_usecase_1, 0, 4)
elif HYPER_PARAMS['use_case'] == 2:
    # use case 2
    X_train, y_train, scalers_train = util.prepare_data(HYPER_PARAMS, train_profiles_usecase_2, 0, 4)
    X_validation, y_validation, scalers_validation = util.prepare_data(HYPER_PARAMS, val_profiles_usecase_2, 0, 4)
    X_test, y_test, scalers_test = util.prepare_data(HYPER_PARAMS, test_profiles_usecase_2, 0, 4)
elif HYPER_PARAMS['use_case'] == 3:
    # use case 3
    X_train, y_train, scalers_train = util.prepare_data(HYPER_PARAMS, train_profiles_usecase_3, 0, 4)
    X_validation, y_validation, scalers_validation = util.prepare_data(HYPER_PARAMS, val_profiles_usecase_3, 0, 4)
    X_test, y_test, scalers_test = util.prepare_data(HYPER_PARAMS, test_profiles_usecase_3, 0, 4)

scalers = scalers_train, scalers_validation, scalers_test 
# scalers shape: ((train_cur, train_volt), (val_cur, val_volt), (test_cur, test_volt))

### Prepare Data for Use Cases

In [ ]:
test_profiles_usecase_1 = ['Profile 10A']
test_profiles_usecase_2 = ['Profile 10A 3x']
test_profiles_usecase_3 = ['Profile -10A']

X_test_1, y_test_1, scalers_test_1 = util.prepare_data(HYPER_PARAMS, test_profiles_usecase_1, 0, 4)
X_test_2, y_test_2, scalers_test_2 = util.prepare_data(HYPER_PARAMS, test_profiles_usecase_2, 0, 4)
X_test_3, y_test_3, scalers_test_3 = util.prepare_data(HYPER_PARAMS, test_profiles_usecase_3, 0, 4)

scalers_usecases = scalers_train, scalers_test_1, scalers_test_2, scalers_test_3 
# scalers shape: ((train_cur, train_volt), (val_cur, val_volt), (test_cur, test_volt))

### Initialize and Train Model

In [ ]:
lstm = lstm.Residual_Model()
lstm.initialize(HYPER_PARAMS)

In [ ]:
_, fig = lstm.train(X_train, y_train, scalers_train)

# save model, hyperparameters and plots
MODEL_ID = str(np.random.randint(10000))
lstm.model.save('trained_models/TGDS/' + str(MODEL_ID))
np.save('trained_models/TGDS/' + str(MODEL_ID) + '/hyperparameters', HYPER_PARAMS)
fig.savefig('trained_models/TGDS/' + str(MODEL_ID) + '/learning_curve.png')

### Test Model

In [ ]:
yhat_train_unscaled, fig = lstm.test(X_train, y_train, scalers)

# save plots and predicted sequences
fig.savefig('trained_models/TGDS/' + str(MODEL_ID) + '/train_profiles.png')
np.save('trained_models/TGDS/' + str(MODEL_ID) + '/predictions', yhat_train_unscaled)

In [ ]:
# lstm.test_usecases(X_train, y_train, X_test_1, y_test_1, X_test_2, y_test_2, X_test_3, y_test_3, scalers_usecases)